In [55]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
import os
from pathlib import Path
import sys
from typing import Optional
if '..' not in sys.path: sys.path.append('..')

import numpy as np
from matplotlib import pyplot as plt
from pydantic_yaml import parse_yaml_file_as
import torch
from torch import nn
from transformers import GPT2Tokenizer, AddedToken, PreTrainedTokenizer

from mllm.config.model import MllmRankerCfg, MllmEncdecCfg, TokenizerCfg
from mllm.data.dsqrels_embs import DsQrelsEmbs, QrelsEmbsBatch
from mllm.data.utils import load_qrels_datasets
from mllm.exp.args import ENCDEC_MODEL_CFG_FNAME, RANKER_MODEL_CFG_FNAME
from mllm.model.mllm_encdec import MllmEncdecLevel
from mllm.model.mllm_ranker import RankProbLoss, MllmRanker, MllmRankerLevel
from mllm.tokenization.chunk_tokenizer import gen_all_tokens, ChunkTokenizer, tokenizer_from_config
from mllm.train.utils import find_create_train_path, calc_print_batches



# Ranker level 1 inference
## Config and paths

In [57]:
DATA_PATH = Path(os.path.expandvars('$HOME')) / 'data'
DS_MSMARCO_DIR_PATH = DATA_PATH / 'msmarco'
DS_FEVER_DIR_PATH = DATA_PATH / 'fever'
TRAIN_RANKER_PATH = DATA_PATH / 'train_mllm_ranker_qrels_0'
TRAIN_RANKER_EMBS_PATH = DATA_PATH / 'train_mllm_ranker_qrels_1'
DS_WIKI_DIR_PATH = DATA_PATH / 'wiki_20200501_en/ch_100_fixed'
DS_EMBS_DIR_PATH = DATA_PATH / 'ranker_embs_msmarco_fever'
CFG_DIR_PATH = Path(os.path.abspath('.')).parent / 'mllm/config/cfg'

tokenizer_cfg_fpath = CFG_DIR_PATH / 'tokenizer_cfg_02.yaml'

ranker0_subdir = 'ranker-20241021_062053-msmarco-fever'
ranker0_train_path = TRAIN_RANKER_PATH / ranker0_subdir
ranker0_snapshot_fpath = ranker0_train_path / 'best.pth'

ranker1_subdir = 'ranker-lvl1-20241023_220614-enc-lrs2-embmatTrue-d256-h8-dec-lrs2-d256-h8-encdec-20241022_224217'
ranker1_subdir = 'ranker-lvl1-20241025_223420-enc-lrs2-embmatTrue-d256-h8-dec-lrs2-d256-h8-encdec-20241022_224217'
ranker1_train_path = TRAIN_RANKER_EMBS_PATH / ranker1_subdir
ranker1_snapshot_fpath = ranker1_train_path / 'best.pth'

ranker0_model_cfg_fpath = ranker0_train_path / RANKER_MODEL_CFG_FNAME
print(f'Ranker cfg fpath: {ranker0_model_cfg_fpath}. Exists: {ranker0_model_cfg_fpath.exists()}')
ranker1_model_cfg_fpath = ranker1_train_path / RANKER_MODEL_CFG_FNAME
print(f'Ranker cfg fpath: {ranker1_model_cfg_fpath}. Exists: {ranker1_model_cfg_fpath.exists()}')

Ranker cfg fpath: /home/misha/data/train_mllm_ranker_qrels_0/ranker-20241021_062053-msmarco-fever/ranker_model_cfg.yaml. Exists: True
Ranker cfg fpath: /home/misha/data/train_mllm_ranker_qrels_1/ranker-lvl1-20241025_223420-enc-lrs2-embmatTrue-d256-h8-dec-lrs2-d256-h8-encdec-20241022_224217/ranker_model_cfg.yaml. Exists: True


In [59]:
emb_chunk_size = 100
embs_chunk_size = 100
docs_batch_size = 10
chunk_size = 100
max_docs_embs = 10
docs_per_chunk = chunk_size // max_docs_embs

device = 'cpu'
# device = 'cuda'

device = torch.device(device)
print(device)
ranker0_model_cfg: MllmRankerCfg = parse_yaml_file_as(MllmRankerCfg, ranker0_model_cfg_fpath)
ranker1_model_cfg: MllmRankerCfg = parse_yaml_file_as(MllmRankerCfg, ranker1_model_cfg_fpath)
enc_cfg_1 = ranker1_model_cfg.encoders[1]


cpu


## Load models and datasets

In [60]:
tkz_cfg = parse_yaml_file_as(TokenizerCfg, tokenizer_cfg_fpath)
ch_tkz = tokenizer_from_config(tkz_cfg)

In [61]:
ds_qrels = load_qrels_datasets([DS_MSMARCO_DIR_PATH, DS_FEVER_DIR_PATH], ch_tkz, emb_chunk_size, device)

Join datasets:
   Msmarco. Queries: 372206. Docs: 3213835. QueryDocRels: 372206
   Fever. Queries: 123142. Docs: 5416568. QueryDocRels: 156101


In [62]:
ds_embs = DsQrelsEmbs(
    ds_dir_path=DS_EMBS_DIR_PATH, chunk_size=embs_chunk_size, emb_size=enc_cfg_1.d_model, emb_dtype=np.float32,
    doc_id_driven=True, max_docs_embs=max_docs_embs, device=device,
)

In [63]:
model_ranker_0 = MllmRankerLevel(ranker0_model_cfg, 0).to(device)
print(f'Loading model weights from {ranker0_snapshot_fpath}')
checkpoint = torch.load(ranker0_snapshot_fpath, map_location=device)
model_ranker_0.load_state_dict(checkpoint['model'])
model_ranker_0.eval()
None

vocab_encoder.src_word_emb.weight (50271, 256) -0.010897174 -1.459274e-06 0.010897168
vocab_encoder.layer_norm.weight (256,) -0.099592425 -0.0026334831 0.09836828
vocab_encoder.layer_norm.bias (256,) -0.09884079 0.0026726224 0.09994449
encoder.a_em () -0.088543944 -0.088543944 -0.088543944
encoder.layer_stack.0.slf_attn.w_qs.weight (256, 256) -0.10824053 0.00010209259 0.10824627
encoder.layer_stack.0.slf_attn.w_ks.weight (256, 256) -0.10825062 0.00021738256 0.108246036
encoder.layer_stack.0.slf_attn.w_vs.weight (256, 256) -0.1082493 -1.4802983e-05 0.10825072
encoder.layer_stack.0.slf_attn.fc.weight (256, 256) -0.10825028 -0.0001302238 0.10824975
encoder.layer_stack.0.slf_attn.layer_norm.weight (256,) -0.09941882 -0.0014857117 0.0993665
encoder.layer_stack.0.slf_attn.layer_norm.bias (256,) -0.09997981 -0.004862113 0.09998429
encoder.layer_stack.0.pos_ffn.w_1.weight (1024, 256) -0.06846507 -4.7036815e-06 0.06846494
encoder.layer_stack.0.pos_ffn.w_1.bias (1024,) -0.099764526 0.00052069005

RuntimeError: Error(s) in loading state_dict for MllmRankerLevel:
	Missing key(s) in state_dict: "decoder.layer_stack.0.slf_attn.w_qs.weight", "decoder.layer_stack.0.slf_attn.w_ks.weight", "decoder.layer_stack.0.slf_attn.w_vs.weight", "decoder.layer_stack.0.slf_attn.fc.weight", "decoder.layer_stack.0.slf_attn.layer_norm.weight", "decoder.layer_stack.0.slf_attn.layer_norm.bias", "decoder.layer_stack.0.pos_ffn.w_1.weight", "decoder.layer_stack.0.pos_ffn.w_1.bias", "decoder.layer_stack.0.pos_ffn.w_2.weight", "decoder.layer_stack.0.pos_ffn.w_2.bias", "decoder.layer_stack.0.pos_ffn.layer_norm.weight", "decoder.layer_stack.0.pos_ffn.layer_norm.bias", "decoder.layer_stack.1.slf_attn.w_qs.weight", "decoder.layer_stack.1.slf_attn.w_ks.weight", "decoder.layer_stack.1.slf_attn.w_vs.weight", "decoder.layer_stack.1.slf_attn.fc.weight", "decoder.layer_stack.1.slf_attn.layer_norm.weight", "decoder.layer_stack.1.slf_attn.layer_norm.bias", "decoder.layer_stack.1.pos_ffn.w_1.weight", "decoder.layer_stac

In [64]:
model_ranker_1 = MllmRankerLevel(ranker1_model_cfg, level=1).to(device)
print(f'Loading model weights from {ranker1_snapshot_fpath}')
checkpoint = torch.load(ranker1_snapshot_fpath, map_location=device)
model_ranker_1.load_state_dict(checkpoint['model'])
model_ranker_1.eval()
None

encoder.layer_stack.0.slf_attn.w_qs.weight (256, 256) -0.108247705 6.458067e-05 0.108241946
encoder.layer_stack.0.slf_attn.w_ks.weight (256, 256) -0.10825289 4.0275627e-06 0.10824833
encoder.layer_stack.0.slf_attn.w_vs.weight (256, 256) -0.1082488 7.5247895e-05 0.10825012
encoder.layer_stack.0.slf_attn.fc.weight (256, 256) -0.10824937 0.0001995811 0.10824907
encoder.layer_stack.0.slf_attn.layer_norm.weight (256,) -0.099848844 0.0021417397 0.099555515
encoder.layer_stack.0.slf_attn.layer_norm.bias (256,) -0.09982987 -0.00022519252 0.09944629
encoder.layer_stack.0.pos_ffn.w_1.weight (1024, 256) -0.06846469 2.8559281e-05 0.068464674
encoder.layer_stack.0.pos_ffn.w_1.bias (1024,) -0.09975264 0.0012859779 0.09967675
encoder.layer_stack.0.pos_ffn.w_2.weight (256, 1024) -0.068464495 7.863473e-05 0.06846463
encoder.layer_stack.0.pos_ffn.w_2.bias (256,) -0.09999155 0.0034040469 0.09917271
encoder.layer_stack.0.pos_ffn.layer_norm.weight (256,) -0.09974629 -0.0009187027 0.099436216
encoder.layer_

### Using dataset queries

In [65]:
embs_view = ds_embs.get_embs_view(batch_size=docs_batch_size * docs_per_chunk)

In [66]:
i_batch = 0
embs_batch_it = embs_view.get_batch_iterator(with_queries=True)
for _ in range(i_batch):
    embs_batch = next(embs_batch_it)
embs_batch = next(embs_batch_it)

In [67]:
assert embs_batch.qs_embs is not None and embs_batch.qs_ind_len is not None
print(f'Batch {i_batch}. Docs embs: {embs_batch.docs_embs.shape}. Queries embs: {embs_batch.qs_embs.shape} {len(embs_batch.qs_ind_len)}')


Batch 0. Docs embs: (10, 100, 256). Queries embs: (149, 256) 149


In [68]:
len(embs_batch.df_docs_ids), len(embs_batch.df_qrels), len(embs_batch.df_qs_ids)

(769, 149, 149)

In [69]:
docs_embs_t = embs_batch.get_docs_embs_tensor()
qs_embs_t, qs_masks_t = embs_batch.get_qs_tensors()
out_rank = model_ranker_1.run_qs_embs(docs_embs_t, qs_embs_t, embs_batch.qs_ind_len)
print(f'out_rank. min, mean, max: {out_rank.min():0.4f}, {out_rank.mean():0.4f}, {out_rank.max():0.4f}')

out_rank. min, mean, max: 0.0912, 0.1000, 0.1095


In [70]:
print(out_rank.shape)

torch.Size([149, 10])


In [71]:
query_ind = 3
dsqid = embs_batch.qs_ind_len[query_ind][0]
print(f'Query {query_ind} {dsqid}: {ds_qrels.df_qs.loc[dsqid].query}')


Query 3 11791: +where does the pacific walrus ive


In [72]:
out_rank[query_ind]

tensor([0.1015, 0.0988, 0.0996, 0.0993, 0.1000, 0.1005, 0.1019, 0.0990, 0.0996,
        0.0999], grad_fn=<SelectBackward0>)

In [73]:
qs_masks_t[query_ind]

tensor([False, False, False, False, False, False, False,  True, False, False])

In [53]:
dsdids = embs_batch.ids.reshape((docs_batch_size, docs_per_chunk))
i_doc = 7
dsdids[i_doc]

array([1369469, 1901654,  632008, 2724055, 2305197, 1019421, 2345255,
       2805343, 3182944,  655573])

In [54]:
for i, dsdid in enumerate(dsdids[i_doc]):
    title, text = ds_qrels.get_doc(dsdid)
    print(f'{i + 1:02d}. Doc {dsdid}: {title[:100]}. {text[:200]}')

01. Doc 1369469: Morse Code & the Telegraph. "Early Forms of Long-Distance Communication Before the development of the electric telegraph in the 19th century revolutionized how information was transmitted across long distances, ancient civilizat
02. Doc 1901654: When Did Mardi Gras Start?. "Listen H urray! It's Tuesday! How often do you hear that? If you've had a bad Monday, then maybe Tuesday rolling around might brighten your spirits. Otherwise, Tuesday usually doesn't get too much at
03. Doc 632008: Walrus. "From Wikipedia, the free encyclopedianavigation search For other uses, see Walrus (disambiguation). Walrus Temporal range: Pleistocene to Recent Male Female with young Conservation status Vulnerable 
04. Doc 2724055: Where Is Bactria?. "Humanities ›History & Culture Where Is Bactria? Share Flipboard Email Printvia Wikipediaby Kallie Szczepanski Updated December 27, 2017Bactria is an ancient region of Central Asia, between the Hindu 
05. Doc 2305197: Find the Routing Number on a 

In [36]:
print(ds_qrels.df_qs.loc[dsqid])
qrel = ds_qrels.df_qrels.loc[dsqid]
print(qrel)

qid                           265
query    +what is fascia or facia
dsid                            1
dsqid                        9174
Name: 9174, dtype: object
qid         265
did       97881
dsid          1
dsqid      9174
dsdid    508390
Name: 9174, dtype: int64


In [38]:
title, text = ds_qrels.get_doc(qrel.dsdid)
print(f'{title[:100]}. {text[:400]}')

fascia. "fascia Also found in: Thesaurus, Medical, Legal, Encyclopedia, Wikipedia. Related to fascia: Colles fasciafas·cia (făsh′ə, fä′shə)n. pl. fas·ci·ae (făsh′ē-ē′, fä′shē-ē′)1. Anatomya. A sheet or band of fibrous connective tissue enveloping, separating, or bindingtogether muscles, organs, and other soft structures of the body.b. The tissue of which such a sheet or band is composed.2. Biology A broad


In [39]:
qrel.dsdid in embs_batch.ids

True

In [35]:
set(ds_qrels.df_qrels.index) == set(ds_qrels.df_qs.index)

True

In [40]:
embs_batch.ids

array([ 895028, 3085932, 1616445, 2396834,  465048,  679478, 1282538,
       2558828, 1918243, 1184395, 1931964,  991350, 1141254, 2247720,
       2726870, 1461296, 2434362,  127982, 1479578, 1000903,  509488,
       1161217, 3026873,  109872, 1446407, 1152228,  598469, 2971199,
       2951585, 1365772, 2635634, 2588840, 2495189,   38604, 1537726,
        699367, 2268633,  517965,  796713,  416362,  299777, 1742059,
        544317, 2520629, 1436264, 2858298,  412377,  456861,  688057,
        826758,  294285, 3081347,  650865, 2410836, 1107353, 3195243,
       1505331, 1627906, 2145371,  739799, 1234207,  508390,  198804,
       1895725,  239631, 1388082, 2056134,   51362,  760521,   38219,
       1369469, 1901654,  632008, 2724055, 2305197, 1019421, 2345255,
       2805343, 3182944,  655573,  308315,  580376, 1664242, 3115493,
       2165184, 2810761,   58275,  525091,  919745, 1647851,  406509,
       1163151, 2521349,  207467, 1206529,  803368,  597726,  874445,
        314351, 1573

In [41]:
for i, did in enumerate(embs_batch.ids):
    if did == 508390:
        print(i)
        break

61


In [42]:
df_off = ds_qrels.df_off
df_off.iloc[:5]

,did,offset,dsid,dsdid
dsdid,,,,
0,1555982,0,1,0
1,301595,1852,1,1
2,1359209,7973,1,2
3,2147834,23656,1,3
4,1568809,31104,1,4


In [43]:
df_off.loc[[1,0,3,2,4]]

,did,offset,dsid,dsdid
dsdid,,,,
1,301595,1852,1,1
0,1555982,0,1,0
3,2147834,23656,1,3
2,1359209,7973,1,2
4,1568809,31104,1,4


In [44]:
df = df_off.loc[[1,0,3,2,4]]
df.index, df.did

(Index([1, 0, 3, 2, 4], dtype='int64', name='dsdid'),
 dsdid
 1     301595
 0    1555982
 3    2147834
 2    1359209
 4    1568809
 Name: did, dtype: int64)

In [ ]:
pad_tok = tkz_cfg.custom_tokens['pad']
inp_len = ranker1_model_cfg.vocab_encoder.inp_len

def tokten_to_text(tokens: torch.Tensor) -> str:
    tokens = tokens.flatten()
    tokens = tokens[tokens != pad_tok]
    tokens = list(tokens)
    s = ch_tkz.decode(tokens)
    return s

def distance(x: np.ndarray, y: np.ndarray, cosine: bool = False):
    if not cosine:
        return np.linalg.norm(x - y)
    x_norm, y_norm = np.linalg.norm(x), np.linalg.norm(y)
    return np.sum(x * y) / (x_norm * y_norm)

def text_to_tokens(s: str, qbeg_tok: Optional[int] = None, qend_tok: Optional[int] = None) -> torch.Tensor:
    tokens = ch_tkz(s)['input_ids']
    if qbeg_tok is not None:
        assert qend_tok is not None
        tokens = [qbeg_tok, *tokens, qend_tok]
    n_tokens = len(tokens)
    n_padded = n_tokens // inp_len + (n_tokens % inp_len > 0)
    res = np.full((n_padded * inp_len, ), pad_tok, dtype=np.int32)
    res[:n_tokens] = tokens
    res = torch.from_numpy(res).to(device)
    res = res.reshape(n_padded, inp_len)
    return res

def print_dist(target_embs: torch.Tensor, docs_embs: torch.Tensor, target_mask: torch.Tensor, cosine: bool = True):
    for i, docs_emb in enumerate(docs_embs.detach().numpy()):
        for target_emb in target_embs.detach().numpy():
            dist = distance(target_emb, docs_emb, cosine)
            print(f'{dist:0.6f} ', end='')
        sfx = 'T' if target_mask[i] else 'F'
        print(sfx)